# Nonlinear regression for KL

In [ ]:
import time
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
dttm = time.strftime("%Y%m%d-%H%M%S")
dttm

In [ ]:
np.random.randint(0x7fff_ffff)

In [ ]:
# random_state = np.random.RandomState(575_727_528)
# random_state = np.random.RandomState()

<br>

## Monte Carlo estimate

In [ ]:
import tqdm

KL-div for $\mathbb{R}$-gaussian vi

$$
KL(\mathcal{N}(w\mid \theta, \alpha \theta^2) \|
        \tfrac1{\lvert w \rvert})
    \propto - \tfrac12 \log \alpha
      + \mathbb{E}_{\xi \sim \mathcal{N}(1, \alpha)}
        \log{\lvert \xi \rvert}
%     = - \tfrac12 \log \alpha
%       + \log{\sqrt{\alpha}}
%       + \tfrac12 \mathbb{E}_{\xi \sim \mathcal{N}(0, 1)}
%         \log{\bigl\lvert \tfrac1{\sqrt{\alpha}} + \xi \bigr\rvert^2}
    = \tfrac12 \mathbb{E}_{\xi \sim \mathcal{N}(0, 1)}
        \log{\bigl\lvert \xi + \tfrac1{\sqrt{\alpha}}\bigr\rvert^2}
    \,. $$

In [ ]:
def kldiv_real_mc(log_alpha, m=1e5):
    kld, m = -0.5 * log_alpha, int(m)
    for i, la in enumerate(tqdm.tqdm(log_alpha)):
        eps = np.random.randn(m) * np.sqrt(np.exp(la))
        kld[i] += np.log(abs(eps + 1)).mean(axis=-1)
    return kld

def kldiv_real_mc_reduced(log_alpha, m=1e5):
    kld, m = np.zeros_like(log_alpha), int(m)
    for i, la in enumerate(tqdm.tqdm(log_alpha)):
        eps = np.random.randn(m)
        kld[i] = np.log(abs(eps + np.exp(-0.5 * la))).mean(axis=-1)
    return kld

Suppose $q(z) = \mathcal{CN}(\theta, \alpha \lvert\theta\rvert^2, 0)$ and
$p(z) \propto \lvert z\rvert^{-\beta}$. Each term in the sum
$$
\begin{align}
    KL(q\|p)
        &= \mathbb{E}_{q(z)} \log \tfrac{q(z)}{p(z)}
        = \mathbb{E}_{q(z)} \log q(z) - \mathbb{E}_{q(z)} \log p(z)
%         \\
%         &= - \log \bigl\lvert \pi e \alpha \lvert\theta\rvert^2 \bigr\rvert
%         + \beta \mathbb{E}_{q(z)} \log \lvert z\rvert
%         + C
%         \\
%         &= - \log \pi e
%         - \log \alpha \lvert\theta\rvert^2
%         + \beta \mathbb{E}_{\varepsilon \sim \mathcal{CN}(1, \alpha, 0)}
%             \log \lvert \theta \rvert \lvert \varepsilon\rvert
%         + C
%         \\
%         &= - \log \pi e - \log \alpha
%         + \tfrac{\beta - 2}2 \log \lvert \theta \rvert^2
%         + \beta \mathbb{E}_{\varepsilon \sim \mathcal{CN}(1, \alpha, 0)}
%             \log \lvert \varepsilon\rvert
%         + C
%         \\
%         &= - \log \pi e - \log \alpha
%         + \tfrac{\beta - 2}2 \log \lvert \theta \rvert^2
%         + \tfrac\beta2 \mathbb{E}_{z \sim \mathcal{CN}(0, \alpha, 0)}
%             \log \bigl\lvert z + 1 \bigr\rvert^2
%         + C
        \\
        &= - \log \pi e
        + \tfrac{\beta - 2}2 \log \lvert \theta \rvert^2
        + \tfrac{\beta-2}2 \log\alpha
        + \tfrac\beta2 \mathbb{E}_{z \sim \mathcal{CN}(0, 1, 0)}
            \log \bigl\lvert z + \tfrac1{\sqrt{\alpha}} \bigr\rvert^2
        + C
        \\
        &= - \log \pi e
        + \tfrac{\beta - 2}2 \log \lvert \theta \rvert^2
        + \tfrac{\beta - 2}2 \log\alpha
        + \tfrac\beta2 \mathbb{E}_{\varepsilon \sim \mathcal{N}_2\bigl(0, \tfrac12 I\bigr)}
            \log \bigl((\varepsilon_1 + \tfrac1{\sqrt{\alpha}})^2 + \varepsilon_2^2\bigr)
        + C
\end{align}
\,. $$

Negative kl-div divergence for $\mathbb{C}$ gaussian and $\beta=2$:
$$
KL(\mathcal{N}^{\mathbb{C}}(w\mid \theta, \alpha \lvert\theta\rvert^2, 0) \|
        \tfrac1{\lvert w \rvert})
    \propto
        - \log\alpha
        + \mathbb{E}_{\xi \sim \mathcal{CN}(1, \alpha, 0)}
            \log \lvert \xi \rvert^2
    = \mathbb{E}_{z \sim \mathcal{CN}(0, 1, 0)}
        \log \bigl\lvert z + \tfrac1{\sqrt{\alpha}} \bigr\rvert^2
    \,. $$

In [ ]:
def kldiv_cplx_mc(log_alpha, m=1e5):
    kld, m = - log_alpha, int(m)
    for i, la in enumerate(tqdm.tqdm(log_alpha)):
        eps = np.random.randn(m) + 1j * np.random.randn(m)
        eps *= np.sqrt(np.exp(la) / 2)
        kld[i] += 2 * np.log(abs(eps + 1)).mean(axis=-1)
    return kld

def kldiv_cplx_mc_reduced(log_alpha, m=1e5):
    kld, m, isq2 = np.zeros_like(log_alpha), int(m), 1/np.sqrt(2)
    for i, la in enumerate(tqdm.tqdm(log_alpha)):
        eps = isq2 * np.random.randn(m) + 1j * isq2 * np.random.randn(m)
        kld[i] = 2 * np.log(abs(eps + np.exp(-0.5 * la))).mean(axis=-1)
    return kld

In fact there is a "simple" expression for the expectation of $
    \log \lvert z + \mu \rvert^2
$ for $z\sim \mathcal{CN}(0, 1, 0)$ found here
[The Expected Logarithm of a Noncentral Chi-Square Random Variable](http://moser-isi.ethz.ch/explog.html)
.

For $u \sim \mathcal{CN}(0, 1, 0)$ and $\mu\in \mathbb{C}$ we have

$$
g(\mu)
    = \mathbb{E} \log \lvert u + \mu\rvert^2
    = \log \lvert \mu \rvert^2 - \mathop{Ei}{(-\lvert \mu \rvert^2)}
    \,, $$

where $
%     \mathop{Ei}(x) = - \int_{-x}^\infty \tfrac{e^{-t}}t dt
    \mathop{Ei}(x) = \int_{-\infty}^x \tfrac{e^u}u du
$.
Thus for $z \sim \mathcal{CN}(0, \alpha, 0)$, $\alpha > 0$, we get

$$
\mathbb{E} \log \lvert z + 1\rvert^2
%     = \mathbb{E} \log \bigl\lvert \sqrt\alpha u + 1\bigr\rvert^2
    = \mathbb{E} \log \alpha \bigl\lvert u + \tfrac1{\sqrt\alpha} \bigr\rvert^2
%     = \log \alpha + g\bigl(\tfrac1{\sqrt\alpha}\bigr)
%     = \log \alpha + \log \tfrac1\alpha - \mathop{Ei}{(-\tfrac1\alpha)}
    = - \mathop{Ei}{(-\tfrac1\alpha)}
    \,. $$

Therefore

$$
\begin{align}
    KL(q\|p)
        &= \mathbb{E}_{q(z)} \log \tfrac{q(z)}{p(z)}
        = \mathbb{E}_{q(z)} \log q(z) - \mathbb{E}_{q(z)} \log p(z)
        \\
        &= - \log \pi e - \log \alpha
        + \tfrac{\beta - 2}2 \log \lvert \theta \rvert^2
        + \tfrac\beta2 \mathbb{E}_{z \sim \mathcal{CN}(0, \alpha, 0)}
            \log \bigl\lvert z + 1 \bigr\rvert^2
        + C
        \\
        &= - \log \pi e - \log \alpha
        + (\beta - 2) \log \lvert \theta \rvert
        - \tfrac\beta2 \mathop{Ei}{(-\tfrac1\alpha)}
        + C
\end{align}
\,. $$

For $\beta = 2$ we get
$$
KL(q\|p)
    = C - \log \pi e - \log \alpha
    - \mathop{Ei}{\bigl(-\tfrac1\alpha \bigr)}
\,. $$

In [ ]:
def kl_div_exact(log_alpha):
    return - log_alpha - expi(- np.exp(- log_alpha))  # - np.euler_gamma

Differentiable exponential integral for torch.

In [ ]:
import torch
import torch.nn.functional as F

from scipy.special import expi

class ExpiFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        ctx.save_for_backward(x)

        x_cpu = x.data.cpu().numpy()
        output = expi(x_cpu, dtype=x_cpu.dtype)
        return torch.from_numpy(output).to(x.device)

    @staticmethod
    def backward(ctx, grad_output):
        x = ctx.saved_tensors[-1]
        return grad_output * torch.exp(x) / x

torch_expi = ExpiFunction.apply

input = torch.randn(20, 20).to(torch.double)
assert torch.autograd.gradcheck(torch_expi, input.requires_grad_(True))

<br>

Loky backend seems to correctly deal with `np.random`:
[Random state within joblib.Parallel](https://joblib.readthedocs.io/en/latest/auto_examples/parallel_random_state.html)

In [ ]:
import joblib

def par_kldiv_real_mc(log_alpha, m=1e5):
    def _kldiv_one(log_alpha):
        eps = np.random.randn(int(m)) * np.sqrt(np.exp(log_alpha))
        return - 0.5 * log_alpha + np.log(abs(eps + 1)).mean(axis=-1)
    kldiv_one = joblib.delayed(_kldiv_one)

    par_ = joblib.Parallel(n_jobs=-1, backend="loky", verbose=0)
    return np.array(par_(kldiv_one(la) for la in tqdm.tqdm(log_alpha)))

Compute (or load from cache) the MC estiamte of the negative Kullback-Leibler divergence.

In [ ]:
import os
import gzip
import joblib

filename = "neg kl-div mc 20190516-134609.gz"
if os.path.exists(filename):
    # load from cache
    with gzip.open(filename, "rb") as fin:
        cache = joblib.load(fin)

    ## HERE!
    neg_kl_real_mc, neg_kl_cplx_mc = cache["real"], cache["cplx"]
    alpha = cache["alpha"]
    log_alpha = np.log(alpha)

else:
    alpha = np.logspace(-8, 8, num=4096)
    log_alpha = np.log(alpha)

    # get an MC estimate of the negative kl-divergence
    neg_kl_real_mc = -kldiv_real_mc(log_alpha, m=1e7)
    neg_kl_cplx_mc = -kldiv_cplx_mc(log_alpha, m=1e7)

    filename = f"neg kl-div mc {dttm}.gz"
    with gzip.open(filename, "wb", compresslevel=5) as fout:
        joblib.dump({
            "m" : 1e7,
            "alpha" : alpha,
            "real": neg_kl_real_mc,
            "cplx": neg_kl_cplx_mc,
        }, fout)
# end if

print(filename)

```text
100%|██████████| 513/513 [16:54<00:00,  1.93s/it]
100%|██████████| 513/513 [39:25<00:00,  4.56s/it]
```

```text
100%|██████████| 4096/4096 [22:08<00:00,  3.25it/s]
100%|██████████| 4096/4096 [56:05<00:00,  1.21it/s]
```

<br>

## Non-linear regression

Negative kl-div approximation from [arxiv:1701.05369](https://arxiv.org/pdf/1701.05369.pdf)

$$
    - KL(\mathcal{N}(w\mid \theta, \alpha \theta^2) \|
            \tfrac1{\lvert w \rvert})
        \approx
            k_1 \sigma(k_2 + k_3 \log \alpha) + C
            - k_4 \log (1 + e^{-\log \alpha})
        \bigg\vert_{C,\, k_4 = -k_1,\, \tfrac12}
    \,. $$

In [ ]:
from scipy.special import expit

def np_neg_kldiv_approx(k, log_alpha):
    k1, k2, k3, k4 = k
    C = 0  # -k1

    sigmoid = expit(k2 + k3 * log_alpha)
    softplus = - k4 * np.logaddexp(0, -log_alpha)
    return k1 * sigmoid + softplus + C

def tr_neg_kldiv_approx(k, log_alpha):
    k1, k2, k3, k4 = k
    C = 0  # -k1

    sigmoid = torch.sigmoid(k2 + k3 * log_alpha)
    softplus = - k4 * F.softplus(- log_alpha)
    return k1 * sigmoid + softplus + C

$x \mapsto \log(1 + e^x)$ is softplus and needs different
 compute paths depending on the sign of $x$:
 $$ x\mapsto \log(1+e^{-\lvert x\rvert}) + \max{\{x, 0\}} \,. $$


$$
    \log\alpha - \log(1 + e^{\log\alpha})
        = \log\tfrac{\alpha}{1 + \alpha}
        = - \log\tfrac{1 + \alpha}{\alpha}
        = - \log(1 + \tfrac1\alpha)
        = - \log(1 + e^{-\log\alpha})
    \,. $$

Fit a curve to the MC estimate: level-grad fused function.

In [ ]:
tr_neg_kl_cplx_mc = torch.from_numpy(neg_kl_cplx_mc)
tr_neg_kl_real_mc = torch.from_numpy(neg_kl_real_mc)
tr_log_alpha = torch.from_numpy(log_alpha)

def fused_mse(k, log_alpha, target):  # torch
    tr_k = torch.from_numpy(k).requires_grad_(True)

    approx = tr_neg_kldiv_approx(tr_k, log_alpha)

    loss = F.mse_loss(approx, target, reduction="mean")
    loss.backward()

    return loss.item(), tr_k.grad.numpy()

<br>

Compare the mc estimate against the exact value

In [ ]:
resid = (- kl_div_exact(log_alpha)) - neg_kl_cplx_mc
plt.plot(log_alpha, resid)

abs(resid).mean(), resid.std()

<br>

Now let's find the optimal nonlinear regresson fit using L-BFGS:
$$
\frac12 \sum_i \bigl(
    y_i - f(\log \alpha_i, \theta)
\bigr)^2 \longrightarrow \min_\theta
    \,. $$

In [ ]:
from scipy.optimize.lbfgsb import fmin_l_bfgs_b

k_real = fmin_l_bfgs_b(fused_mse, np.r_[0.5, 0., 1., 0.5],
                       bounds=((None, None), (None, None), (None, None), (0.5, 0.5)),
                       args=(tr_log_alpha, tr_neg_kl_real_mc))[0]

k_cplx = fmin_l_bfgs_b(fused_mse, np.r_[0.5, 0., 1., 1.],
                       bounds=((None, None), (None, None), (None, None), (1.0, 1.0)),
                       args=(tr_log_alpha, tr_neg_kl_cplx_mc))[0]

The fit coefficients from the paper.

In [ ]:
k_real_1701_05369 = np.r_[0.63576, 1.87320, 1.48695, 0.5]

In [ ]:
k_real_1701_05369, k_real, k_cplx

```text
(array([0.63576, 1.8732 , 1.48695, 0.5    ]),
 array([0.63567313, 1.88114543, 1.49136378, 0.5       ]),
 array([0.57810091, 1.45926293, 1.36525956, 1.        ]))
```

<br>

In [ ]:
neg_kl_real_1701_05369 = np_neg_kldiv_approx(k_real_1701_05369, log_alpha)
neg_kl_real_approx = np_neg_kldiv_approx(k_real, log_alpha)
neg_kl_cplx_approx = np_neg_kldiv_approx(k_cplx, log_alpha)

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111, xlabel=r"$\log\alpha$", ylabel="-kld")

ax.plot(log_alpha, neg_kl_real_mc - neg_kl_real_1701_05369, label=r"$\mathbb{R}$ - arXiv:1701.05369", alpha=0.5)
ax.plot(log_alpha, neg_kl_real_mc - neg_kl_real_approx, label=r"$\mathbb{R}$ - lbfgs", alpha=0.5)
ax.plot(log_alpha, neg_kl_cplx_mc - neg_kl_cplx_approx, label=r"$\mathbb{C}$ - lbfgs")

ax.axhline(0., c="k", zorder=-10)
ax.legend(ncol=3)
ax.set_title("Regression residuals of the MC estimate of the KL-div for ARD")
plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111, xlabel=r"$\log\alpha$", ylabel="-kld")

ax.plot(log_alpha, -kl_div_exact(log_alpha), c="k", label=r"$\mathbb{C}$ - exact", lw=2)

ax.plot(log_alpha, neg_kl_real_mc, label=r"$\mathbb{R}$")
ax.plot(log_alpha, neg_kl_cplx_mc, label=r"$\mathbb{C}$")

ax.legend(ncol=3)
ax.set_title("the MC estimate of the KL-div for ARD")
plt.show()

Indeed, really close in unform norm ($\|\cdot\|_\infty$ on $C^1(\mathbb{R})$).

In [ ]:
abs(neg_kl_real_mc - neg_kl_real_1701_05369).max(), \
abs(neg_kl_real_mc - neg_kl_real_approx).max(), \
abs(neg_kl_cplx_mc - neg_kl_cplx_approx).max()

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111, xlabel=r"$\log\alpha$", ylabel="-kld")

ax.plot(log_alpha, neg_kl_cplx_mc, label=r"$\mathbb{C}$ - mc")
ax.plot(log_alpha, neg_kl_cplx_approx, label=r"$\mathbb{C}$ - approx")

ax.plot(log_alpha, neg_kl_real_mc, label=r"$\mathbb{R}$ - mc")
ax.plot(log_alpha, neg_kl_real_approx, label=r"$\mathbb{R}$ - approx")

# ax.set_xlim(0, 20)
# ax.set_ylim(0.55, 0.6)

ax.legend()

## Exact kl-div grad for $\mathbb{R}$-gaussian (draft)

Note this paper here
[Moments of the log non-central chi-square distribution](https://arxiv.org/pdf/1503.06266.pdf)
correctly notices that on
[p. 2446 of Lapidoth, Moser (2003)](http://moser-isi.ethz.ch/docs/papers/alap-smos-2003-3.pdf)
there is a missing $\log{2}$ term. In the following analysis resembles the logic of this paper. 

Let $(z_i)_{i=1}^m \sim \mathcal{N}(0, 1)$ iid and $
  (\mu_i)_{i=1}^m \in \mathbb{R}
$. The random variable $
  W = \sum_i (\mu_i + z_i)^2
$ is said to be $\chi^2_m(\lambda)$ distributed (noncentral $\chi^2$)
with noncentrality parameter $\lambda = \sum_i \mu_i^2$.

Consider the mgf of $W$:

$$
M_W(t)
    = \mathbb{E}(e^{Wt})
    = \prod_i \mathbb{E}(e^{(\mu_i + z_i)^2 t})
    \,, $$

by independence. Now for $z \sim \mathcal{N}(\mu, 1)$

$$
\mathbb{E}(e^{z^2 t})
    = \tfrac1{\sqrt{2\pi}}
    \int_{-\infty}^{+\infty}
        e^{z^2 t} e^{-\tfrac{(z-\mu)^2}2}
    dz
    \,. $$

Now, for $t < \tfrac12$

$$
z^2 t - \tfrac{(z - \mu)^2}2
    = - \tfrac12 (1 - 2t) z^2 + z \mu - \tfrac{\mu^2}2
%     = - \tfrac12 (1 - 2t) \bigl(
%         z^2 - 2 z \tfrac\mu{1 - 2t} + \tfrac{\mu^2}{1 - 2t}
%     \bigr)
%     = - \tfrac12 (1 - 2t) \bigl( z - \tfrac\mu{1 - 2t} \bigr)^2
%     - \tfrac12 (1 - 2t) \bigl(
%         \tfrac{\mu^2}{1 - 2t}
%         - \tfrac{\mu^2}{(1 - 2t)^2}
%     \bigr)
%     = - \tfrac12 (1 - 2t) \bigl( z - \tfrac\mu{1 - 2t} \bigr)^2
%     - \tfrac{\mu^2}2 \bigl(
%         \tfrac{1 - 2t}{1 - 2t} - \tfrac1{1 - 2t}
%     \bigr)
%     = - \tfrac12 (1 - 2t) \bigl( z - \tfrac\mu{1 - 2t} \bigr)^2
%     + \tfrac{\mu^2}2 \tfrac{2t}{1 - 2t}
    = - \tfrac12 (1 - 2t) \bigl( z - \tfrac\mu{1 - 2t} \bigr)^2
    + \mu^2 \tfrac{t}{1 - 2t}
    \,, $$

whence

$$
\mathbb{E}(e^{z^2 t})
    = \tfrac1{\sqrt{2\pi}}
    \int_{-\infty}^{+\infty}
        e^{z^2 t} e^{-\tfrac{(z-\mu)^2}2}
    dz
%     = e^{\mu^2 \tfrac{t}{1 - 2t}}
%     \tfrac1{\sqrt{2\pi}}
%     \int_{-\infty}^{+\infty}
%         e^{- \tfrac12 (1 - 2t) \bigl( z - \tfrac\mu{1 - 2t} \bigr)^2}
%     dz
    = e^{\mu^2 \tfrac{t}{1 - 2t}}
    \tfrac1{\sqrt{2\pi}}
    \int_{-\infty}^{+\infty}
        e^{- \tfrac12 (1 - 2t) z^2}
    dz
%     = [u = \sqrt{1 - 2t} z]
%     = e^{\mu^2 \tfrac{t}{1 - 2t}}
%     \tfrac1{\sqrt{2\pi}}
%     \int_{-\infty}^{+\infty}
%         e^{- \tfrac12 u^2}
%     \tfrac{du}{\sqrt{1 - 2t}}
    = \tfrac{
        \exp{\{\mu^2 \tfrac{t}{1 - 2t}\}}
    }{\sqrt{1 - 2t}}
    \,. $$

Therefore 

$$
M_W(t)
    = \mathbb{E}(e^{Wt})
    = \prod_i \tfrac{
        e^{\mu_i^2 \tfrac{t}{1 - 2t}}
    }{\sqrt{1 - 2t}}
%     = e^{\lambda \tfrac{t}{1 - 2t}}
%     (1 - 2t)^{-\tfrac{m}2}
%     = e^{\lambda \tfrac{- t}{2t - 1}}
%     (1 - 2t)^{-\tfrac{m}2}
%     = e^{\tfrac\lambda2 \tfrac{1 - 2t - 1}{2t - 1}}
%     (1 - 2t)^{-\tfrac{m}2}
%     = e^{- \tfrac\lambda2 (1 + \tfrac1{2t - 1})}
%     (1 - 2t)^{-\tfrac{m}2} 
    = e^{- \tfrac\lambda2} e^{\tfrac\lambda2 \tfrac1{1 - 2t}}
    (1 - 2t)^{-\tfrac{m}2}
    \,. $$

Expanding the exponential as infinte series:

$$
M_W(t)
    = e^{- \tfrac\lambda2} e^{\tfrac\lambda2 \tfrac1{1 - 2t}}
    (1 - 2t)^{-\tfrac{m}2}
%     = e^{- \tfrac\lambda2} (1 - 2t)^{-\tfrac{m}2}
%     \sum_{n \geq 0} \tfrac{\bigl(\tfrac\lambda2\bigr)^n}{n! (1 - 2t)^n}
    = \sum_{n \geq 0} \tfrac{e^{- \tfrac\lambda2} \bigl(\tfrac\lambda2\bigr)^n}{n!}
        (1 - 2t)^{-\tfrac{2n + m}2}
    = \sum_{n \geq 0} \tfrac{e^{- \tfrac\lambda2} \bigl(\tfrac\lambda2\bigr)^n}{n!}
        \mathbb{E}_{x \sim \chi^2_{m + 2n}}(e^{x t})
    \,. $$

<br>

Thus (really? this is how we derive this?!) the density of a non-central $\chi^2_m(\lambda)$ is given by

$$
f_W(x)
    = e^{- \tfrac\lambda2} \sum_{n \geq 0} \tfrac{\bigl(\tfrac\lambda2\bigr)^n}{n!}
        \tfrac{
            x^{n + \tfrac{m}2 - 1} e^{-\tfrac{x}2}
        }{
            2^{n + \tfrac{m}2} \Gamma(n + \tfrac{m}2)
        }
%     = e^{- \tfrac\lambda2} \sum_{n \geq 0} \tfrac{\bigl(\tfrac\lambda2\bigr)^n}{n!}
%         \tfrac{
%             \bigl(\tfrac{x}2\bigr)^{n + \tfrac{m}2 - 1} e^{-\tfrac{x}2}
%         }{
%             2 \Gamma(n + \tfrac{m}2)
%         }
    = \frac12 e^{- \tfrac{x + \lambda}2} \bigl(\tfrac{x}2\bigr)^{\tfrac{m}2 - 1}
    \sum_{n \geq 0} \tfrac{
            \bigl(\tfrac{x \lambda}4\bigr)^n
        }{
            n! \Gamma(n + \tfrac{m}2)
        }
%     = \frac12 e^{- \tfrac{x + \lambda}2}
%     \bigl(\tfrac{x}\lambda\bigr)^{\tfrac{m}4 - \tfrac12}
%     \bigl(\tfrac{x \lambda}4\bigr)^{\tfrac{m}4 - \tfrac12}
%     \sum_{n \geq 0} \tfrac{
%             \bigl(\tfrac{x \lambda}4\bigr)^n
%         }{
%             n! \Gamma(n + \tfrac{m}2)
%         }
%     = \frac12 e^{- \tfrac{x + \lambda}2}
%     \bigl(\tfrac{x}\lambda\bigr)^{\tfrac{m}4 - \tfrac12}
%     \bigl(\tfrac{\sqrt{x \lambda}}2\bigr)^{\tfrac{m}2 - 1}
%     \sum_{n \geq 0} \tfrac{
%             \bigl(\tfrac{x \lambda}4\bigr)^n
%         }{
%             n! \Gamma(n + \tfrac{m}2)
%         }
    = \frac12 e^{- \tfrac{x + \lambda}2}
    \bigl(\tfrac{x}\lambda\bigr)^{\tfrac{m - 2}4}
    I_{\bigl(\tfrac{m}2 - 1\bigr)}(\sqrt{x \lambda})
\,, $$

where $I_k$ is the modified Bessel function of the first kind

$$
I_k(s)
    = \Bigl(\frac{s}2\Bigr)^k
        \sum_{n \geq 0} \tfrac{
            \bigl( \tfrac{s}2 \bigr)^{2n}
        }{n! \Gamma(n + k + 1)}
    \,. $$

The expected logarithm of $W$ is
$$
\mathbb{E}_{W\sim \chi^2_m(\lambda)} \log W
    = \int_0^\infty f_W(x) \log x dx
%     = \frac12 e^{- \tfrac\lambda2}
%     \int_0^\infty \sum_{n \geq 0} \biggl(
%         \tfrac{\bigl(\tfrac\lambda2\bigr)^n}{n!}
%         \tfrac{e^{- \tfrac{x}2}}{\Gamma(n + \tfrac{m}2)}
%         \bigl(\tfrac{x}2\bigr)^{n + \tfrac{m}2 - 1}
%     \biggr) \log x dx
%     = [\text{ absolute summability and Fubini, or any other conv. thm for integrals of non-negative integrals}]
%     = e^{- \tfrac\lambda2} \sum_{n \geq 0}
%         \tfrac{\bigl(\tfrac\lambda2\bigr)^n}{n!}
%         \tfrac1{\Gamma(n + \tfrac{m}2)}
%     \int_0^\infty
%         e^{- \tfrac{x}2} \bigl(\tfrac{x}2\bigr)^{n + \tfrac{m}2 - 1}
%         (\log2 + \log \tfrac{x}2) \tfrac{dx}2
%     = [u = \tfrac{x}2]
%     = e^{- \tfrac\lambda2} \sum_{n \geq 0}
%         \tfrac{\bigl(\tfrac\lambda2\bigr)^n}{n!}
%     \tfrac1{\Gamma(n + \tfrac{m}2)}
%     \int_0^\infty
%         e^{- u} u^{n + \tfrac{m}2 - 1}
%         (\log2 + \log u) du
%     = [\text{definitions:} \Gamma, \psi]
%     = e^{- \tfrac\lambda2} \sum_{n \geq 0}
%         \tfrac{\bigl(\tfrac\lambda2\bigr)^n}{n!}
%     (\psi(n + \tfrac{m}2) + \log2)
    = \log{2}
    + e^{- \tfrac\lambda2} \sum_{n \geq 0}
        \tfrac{\bigl(\tfrac\lambda2\bigr)^n}{n!}
        \psi(n + \tfrac{m}2)
  \,, $$

<br>

Turns out the expectation $
  \mathbb{E}_{z \sim \mathcal{N}(0, 1)}
    \log{\bigl\lvert z + \tfrac1{\sqrt{\alpha}} \bigr\rvert^2}
$ is equal to $
  \log{2} + g_1(\tfrac1{2\alpha})
$ where

$$
g_m(x)
%     = e^{-x} \sum_{n \geq 0} \frac{x^n}{n! \Gamma(n + \tfrac{m}2)}
%         \int_0^\infty e^{-t} t^{n + \tfrac{m}2-1} \log{t} dt
% e^{-x} \sum_{n=0}^{\infty} \frac{x^n}{n!} \psi(n + m / 2)
    = e^{-x} \sum_{n \geq 0} \frac{x^n}{n!} \psi(n + \tfrac{m}2)
    \,, $$

and $\psi(x)$ is the digamma function, i.e. $
    \psi(x) = \tfrac{d}{dx} \Gamma(x)
$. The digamma function has the following useful properties:
$
    \psi(z+1) = \psi(z) + \tfrac1z
$ and $
    \psi(\tfrac12) = -\gamma - 2\log 2
$.

Differentiating the series within $g_m$ yields:
$$
\frac{d}{d x} g_m(x)
    = e^{-x} \sum_{n\geq 1} \frac{x^{n-1}}{(n-1)!} \psi(n + \tfrac{m}2) - g_m(x)
%     = e^{-x} \sum_{n\geq 0} \frac{x^n}{n!} \psi(n + \tfrac{m}2 + 1) - g_m(x)
%     = e^{-x} \sum_{n\geq 0} \frac{x^n}{n!} (
%         \psi(n + \tfrac{m}2 + 1) - \psi(n + \tfrac{m}2)
%     )
%     = e^{-x} \sum_{n\geq 0} \frac{x^n}{n!} \tfrac1{n + \tfrac{m}2}
    = e^{-x} \sum_{n\geq 0} \frac{x^n}{n!} (n + \tfrac{m}2)^{-1}
    = \cdots
    \,. $$

We can differentiate the series in $g_m(x)$, since the sum converges everywhere
on $\mathbb{R}$. Indeed, it is a power series featuring nonegative coefficients,
which is dominated by $
  \sum_{n \geq 1} \tfrac{x^n}{n!} \log{(n+\tfrac{m}2)}
$, because $\psi(x)\leq \log x - \tfrac1{2x}$. By the ratio test, the dominating
series has infinite radius:

$$
\lim_{n\to\infty}
    \biggl\lvert
        \frac{
            n! x^{n+1} \log{(n + 1 + \tfrac{m}2)}
        }{
            x^n \log{(n + \tfrac{m}2)} (n+1)!
        }
    \biggr\rvert
    = \lim_{n\to\infty}
        \lvert x \rvert 
        \biggl\lvert
            \frac{
                \log{(n + 1 + \tfrac{m}2)}
            }{
                \log{(n + \tfrac{m}2)} (n+1)
            }
        \biggr\rvert
%     = \lim_{n\to\infty}
%         \lvert x \rvert 
%         \biggl\lvert
%             \frac{n + \tfrac{m}2}{
%                 (n + 1 + \tfrac{m}2)((n+1) + (n + \tfrac{m}2) \log{(n + \tfrac{m}2)})
%             }
%         \biggr\rvert
    = 0 < 1
    \,. $$

Since
$$
\frac{\log{x + a + 1}}{x \log{x + a}}
    \sim \frac{\log{x+1}}{(x-a) \log{x}}
    \sim \frac{\log{x+1}}{x \log{x}}
    \sim \frac{\tfrac1{x+1}}{1 + \log{x}}
    \to 0
    \,. $$

A theroem from calculus states, that the formal series derivative (integral)
coincides with the derivative (integral) of the function, corresponding to
the power series (everywhere on the convergence region). And the convergence
regions of derivative (integral) concide with the region of the original
power series.

Now, observe that $
    e^t  = \sum_{n\geq 0} \tfrac{t^n}{n!}
$ on $\mathbb{R}$, for $\alpha\neq 0$ we have $
    \int_0^x t^{\alpha-1} dt
        = \tfrac{x^\alpha}{\alpha}
$ and that

$$
\sum_{n\geq 0} \int_0^x \frac{t^{n+\alpha-1}}{n!} dt
    = \int_0^x \sum_{n\geq 0} \frac{t^{n+\alpha-1}}{n!} dt
    = \int_0^x t^{\alpha-1} e^t dt
    \,. $$

by (MCT) on $
    ([0, x], \mathcal{B}([0, x]), dx)
$ whence

$$
\cdots
    = x^{-\tfrac{m}2} e^{-x} \sum_{n\geq 0}
        \frac{x^{n + \tfrac{m}2}}{n!} (n + \tfrac{m}2)^{-1}
%     = x^{-\tfrac{m}2} e^{-x} \sum_{n\geq 0}
%         \frac1{n!} \int_0^x t^{n + \tfrac{m}2 - 1} dt
%     = x^{-\tfrac{m}2} e^{-x}
%         \int_0^x t^{\tfrac{m}2 - 1} \sum_{n\geq 0} \frac{t^n}{n!} dt
    = x^{-\tfrac{m}2} e^{-x}
        \int_0^x t^{\tfrac{m}2 - 1} e^t dt
    = \cdots
    \,. $$

Using $u^2 = t$ on $[0, \infty]$ we get $2u du = dt$ and 

$$
\int_0^x t^{\tfrac{m}2 - 1} e^t  dt
%     = \int_0^{\sqrt{x}} u^{m - 2} e^{u^2} 2 u du
    = 2 \int_0^{\sqrt{x}} u^{m - 1} e^{u^2} du
    \,.$$

Therefore

$$
\frac{d}{d x} g_m(x)
%     = x^{-\tfrac{m}2} e^{-x}
%         \int_0^x t^{\tfrac{m}2 - 1} e^t dt
    = 2 x^{-\tfrac{m}2} e^{-x}
        \int_0^{\sqrt{x}} u^{m - 1} e^{u^2} du
    \,. $$

For $m=1$

$$
g_1(x)
    = - \gamma - 2 \log{2}
    + e^{-x} \sum_{n\geq 0} \frac{x^n}{n!} \sum_{p=1}^n \frac2{2p - 1}
    \,, $$

and the derivative can be computed thus 

$$
\frac{d}{d x} g_1(x)
    = 2 \tfrac{F(\sqrt{x})}{\sqrt{x}}
    \,, $$

using Dawson's integral, $
    F\colon \mathbb{R} \to \mathbb{R}
    \colon x \mapsto e^{-x^2} \int_0^x e^{u^2} du
$, which exists as a special function (in `scipy`).

Now in SGD (and specifically in SGVB) we are concerned more with
the gradient field, induced by the potential (which is the loss
function), rather than the value itself. Thus, as far as the regularizing
penalty term is concerned, which is used to regularize the loss
objective, we can essentially ignore it's forward pass value (level)
and just compute its gradient (subgradient, normal cone) with respect
the parameter of interest in sgd. (unless it is a part of a constraint,
ie. downstream computation)

The derivative wrt $\alpha$ is 
$$
\tfrac{d}{d \alpha} g_1(\tfrac1{2\alpha})
    = -\tfrac1{2 \alpha^2} g_1'(\tfrac1{2\alpha})
%     = -\tfrac1{2 \alpha^2} 2 \tfrac{F(\sqrt{\tfrac1{2\alpha}})}{\sqrt{\tfrac1{2\alpha}}}
%     = -\tfrac1{2 \alpha^2} 2 \tfrac{F(\tfrac1{\sqrt{2\alpha}})}{\tfrac1{\sqrt{2\alpha}}}
    = -\tfrac1{\alpha} \sqrt{\tfrac2{\alpha}} F(\tfrac1{\sqrt{2\alpha}})
    \,. $$

Since $\alpha$ is nonegative, it it typically parametereized through its
logarithm and computed when needed. Thus in particular the gradient of
the divergence penalty w.r.t $\log \alpha$ is
$$
\frac{d}{d\log \alpha}
    \tfrac12 \mathbb{E}_{z \sim \mathcal{N}(0,1)}
    \log \lvert z + \tfrac1{\sqrt{\alpha}} \rvert^2
    = \frac12 \frac{d\alpha}{d\log \alpha}
        \frac{d}{d\alpha} \bigl(\mathbb{E}\cdots \bigr)
    = - \frac\alpha2 \tfrac1{\alpha}
        \sqrt{\tfrac2{\alpha}} F(\tfrac1{\sqrt{2\alpha}})
    = - \tfrac1{\sqrt{2\alpha}} F(\tfrac1{\sqrt{2\alpha}})
    \,. $$

In [ ]:
from scipy.special import dawsn

def kl_real_deriv(log_alpha):
    tmp = np.exp(- 0.5 * (log_alpha + np.log(2)))
    return -dawsn(tmp) * tmp

For $\beta = 2$ we get
$$
KL(q\|p)
    = C - \log \pi e - \log \alpha
    - \mathop{Ei}{\bigl(-\tfrac1\alpha \bigr)}
\,. $$

In [ ]:
def kl_cplx_deriv(log_alpha):
    return -1 + np.exp(-np.exp(-log_alpha))

$$
\frac{d}{d y} \mathop{Ei}(-e^{-y})
    = \frac{d}{d x} \mathop{Ei}(x) \bigg\vert_{x=-e^{-y}}
    \frac{d(-e^{-y})}{d y}
    = \frac{e^x}{x} \bigg\vert_{x=-e^{-y}} e^{-y}
    = - e^{-e^{-y}}
    \,. $$

Use autograd to get the derivative of the approximation

In [ ]:
def neg_kldiv_approx_deriv(log_alpha, k):
    k, x = map(torch.from_numpy, (k, log_alpha))

    x.requires_grad_(True)
    kldiv = tr_neg_kldiv_approx(k, x).sum()
    grad, = torch.autograd.grad(kldiv, x, grad_outputs=torch.tensor(1.).to(x))

    return log_alpha, grad.cpu().numpy()

Let's estimate the derivative wrt $\log\alpha$ using symmetric differences.

In [ ]:
def symm_diff(x, y):
    """Symmetric difference derivative approximation.

    Assumes x_i is sorted (axis=0), y_i = y(x_i).
    """
    return x[1:-1], (y[2:] - y[:-2]) /(x[2:] - x[:-2])

Darken a given colour.

In [ ]:
from matplotlib.colors import to_rgb
from colorsys import rgb_to_hls, hls_to_rgb

def darker(color, a=0.5):
    """Adapted from this stackoverflow question_.
    .. _question: https://stackoverflow.com/questions/37765197/
    """

    h, l, s = rgb_to_hls(*to_rgb(color))
    return hls_to_rgb(h, max(0, min(a * l, 1)), s)

Plot the `numerical` derivative of the MC estimate, the exact derivative and
the derivative of the fit approximation.

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111, xlabel=r"$\log\alpha$", ylabel="-kld")

# plot kl-real
line, = ax.plot(*neg_kldiv_approx_deriv(log_alpha, k_real_1701_05369),
                label=r"$\partial {KL}_\mathbb{R}$-approx",
                linestyle="-", c="red")

color, zorder = darker(line.get_color(), .75), line.get_zorder()
ax.plot(log_alpha, -kl_real_deriv(log_alpha),
        label=r"$\partial {KL}_\mathbb{R}$-exact",
        linestyle="--", c=color, zorder=zorder + 5, lw=3, alpha=0.5)

color = darker(line.get_color(), 1.5)
ax.plot(*symm_diff(log_alpha, neg_kl_real_mc),
        label=r"$\Delta {KL}_\mathbb{R}$-MC",
        c=color, zorder=zorder - 5, alpha=0.5)


# plot kl-cplx
line, = ax.plot(*neg_kldiv_approx_deriv(log_alpha, k_cplx),
                label=r"$\partial {KL}_\mathbb{C}$-approx",
                linestyle="-", c="blue")

color, zorder = darker(line.get_color(), .75), line.get_zorder()
ax.plot(log_alpha, -kl_cplx_deriv(log_alpha),
        label=r"$\partial {KL}_\mathbb{C}$-exact",
        linestyle="--", c=color, zorder=zorder + 5, lw=3, alpha=0.5)

color = darker(line.get_color(), 1.5)
ax.plot(*symm_diff(log_alpha, neg_kl_cplx_mc),
        label=r"$\Delta {KL}_\mathbb{C}$-MC",
        c=color, zorder=zorder - 5, alpha=0.5)

# ax.axhline(0., c="k", zorder=-10, alpha=0.15)
# ax.axhline(1., c="k", zorder=-10, alpha=0.15)
ax.legend(ncol=2)

ax.set_xlim(-8, +8)

fig.savefig("./assets/grad_log.pdf", dpi=300, format="pdf")
plt.show()

Absolute difference for the exact and approximation's derivative.

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111, xlabel=r"$\log\alpha$", ylabel="-kld")

ax.plot(log_alpha, abs(
        kldiv_approx_deriv(log_alpha, k_real_1701_05369)
        - (-kl_real_deriv(log_alpha))
    ), label=r"$\partial {KL}_\mathbb{R}$: approx - exact")

ax.plot(log_alpha, abs(
        kldiv_approx_deriv(log_alpha, k_cplx)
        - (-kl_cplx_deriv(log_alpha))
    ), label=r"$\partial {KL}_\mathbb{C}$: approx - exact")

ax.axhline(0., c="k", zorder=-10)
ax.legend(ncol=2)

plt.show()

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111, xlabel=r"$\log\alpha$", ylabel="-kld")

# mid_log_alpha = (log_alpha[1:] + log_alpha[:-1]) / 2
# d_log = log_alpha[1:] - log_alpha[:-1]

ax.plot(*symm_diff(log_alpha, neg_kl_real_mc), alpha=0.5,
        label=r"$\partial {KL}_\mathbb{R}$-MC symm.d")
ax.plot(log_alpha, -kl_real_deriv(log_alpha), label=r"$\partial {KL}_\mathbb{R}$-exact")

ax.plot(*symm_diff(log_alpha, neg_kl_cplx_mc), alpha=0.5,
        label=r"$\partial {KL}_\mathbb{C}$-MC symm.d")
ax.plot(log_alpha, -kl_cplx_deriv(log_alpha), label=r"$\partial {KL}_\mathbb{C}$-exact")

ax.axhline(0., c="k", zorder=-10)
ax.legend(ncol=2)

plt.show()

In [ ]:
assert False, """STOP!"""

In [ ]:
from scipy.special import erf, expit

x = np.linspace(-10, 10, num=513)
plt.plot(x, 1 - expit(x))
plt.plot(x, 1 - (erf(x) + 1) / 2)

<br>

## Draft

Consider a complex random vector $z \in \mathbb{C}^d$
$$
    z \sim \mathcal{CN}_d \bigl(\theta, K, C \bigr)
        \Leftrightarrow
        \begin{pmatrix}\Re z \\ \Im z\end{pmatrix}
            \sim \mathcal{N}_{2 d} \biggl(
                \begin{pmatrix}\Re \theta \\ \Im \theta \end{pmatrix},
                \tfrac12 \begin{pmatrix}
                     \Re (K + C) & - \Im (K - C) \\
                     \Im (K + C) & \Re (K - C)
                \end{pmatrix}
            \biggr)
    \,. $$

If $x \sim \mathcal{N}_{2n}\Bigl( \mu, \Sigma\Bigr)$ with $x = (x_1, x_2)$, then $z = x_1 + i x_2$
is a complex gaussian random vector,  $z \sim \mathcal{CN}_n(\mu_1 + i \mu_2, K, C)$ with
$$
\begin{align}
    K &= \Sigma_{11} + \Sigma_{22} + i (\Sigma_{21} - \Sigma_{12})
        \,, \\
    C &= \Sigma_{11} - \Sigma_{22} + i (\Sigma_{21} + \Sigma_{12})
        \,.
\end{align}
$$
Note that $\Sigma_{12} = \Sigma_{21}^\top$, and $\Sigma_{11}, \Sigma_{22} \succeq 0$ imply that
$K\succeq 0$, $K^H = K$, $C = C^\top$, and $\overline{\Gamma} \succeq \overline{C} \Gamma^{-1} C $.

If $A\in \mathbb{C}^{n\times d}$ and $b\in \mathbb{C}^n$, then
$$
    A z + b \sim \mathcal{CN}_n \bigl(A \theta + b, A K A^H, A C A^\top \bigr)
    \,. $$

Indeed,
$$
    \mathbb{E} Az + b = A \mathbb{E} z + b = A \theta + b
    \,, $$
and for $\xi = Az - A\theta$ we have
$$
    \mathbb{E} A (z-\theta)(z-\theta)^H A^H = A K A^H
    \,,
    \mathbb{E} A (z-\theta)(z-\theta)^\top A^\top = A C A^\top
    \,. $$

A complex vector is called _proper_ (or circularly-symmetric) iff the pseudo-covariance
matrix $C$ vanishes. This means that the corresponding blocks in the double-vector real
representation obey $\Sigma_{11} = \Sigma_{22}$ and $\Sigma_{21} = - \Sigma_{12}$. This
last condition implies that $\Sigma_{12}$ is skew-symmetric: $\Sigma_{12} = -\Sigma_{12}^\top$.
This also means that the real and imaginary components of each element of $z$ are
uncorrelated, since skew-symmetry means that its main diagonal is zero.

<br>

### Output of a random complex-linear function

Consider $y = Wx = (I \otimes x^\top) \mathop{vec}(W)$ for
$$
    \mathop{vec}(W) \sim \mathcal{CN}_{[d_1\times d_0]}
        \Bigl(\mathop{vec} \theta, K, C\Bigr)
    \,, $$
for $K \in \mathbb{C}^{[d_1\times d_0]\times [d_1\times d_0]}$ diagonal $K = \mathop{diag}(k_\omega)$.

Since $K = K^H$ we must have $k_\omega = \overline{k_\omega}$, whence $k_\omega \in \mathbb{R}$
and $k_\omega \geq 0$. The relation matrix $C$ is also diagonal with $c_\omega \in \mathbb{C}$.

Observe that for $A = I \otimes x^\top$ we have $A^H = I \otimes \bar{x}$
and $A^\top = I \otimes x$ both $[d_1\times d_0]\times [d_1\times 1]$ matrices.
$$
    A K A^H
        = \sum_{i=1}^{d_1} \sum_{j=1}^{d_0}
            A (e_i \otimes e_j) k_{ij} (e_i \otimes e_j)^\top A^H
        = \sum_{i=1}^{d_1} \sum_{j=1}^{d_0} e_i e_i^\top x_j k_{ij} \bar{x}_j
        = \sum_{i=1}^{d_1} e_i e_i^\top \Bigl\{ \sum_{j=1}^{d_0} k_{ij} x_j \bar{x}_j \Bigr\}
    \,, $$
and
$$
    A C A^\top
        = \sum_{i=1}^{d_1} \sum_{j=1}^{d_0}
            A (e_i \otimes e_j) c_{ij} (e_i \otimes e_j)^\top A^\top
        = \sum_{i=1}^{d_1} \sum_{j=1}^{d_0} e_i e_i^\top x_j c_{ij} x_j
        = \sum_{i=1}^{d_1} e_i e_i^\top \Bigl\{ \sum_{j=1}^{d_0} c_{ij} x_j x_j \Bigr\}
    \,, $$

Therefore
$$
    y \sim \mathcal{CN}_{d_1}\Bigl(
        \theta x, \sum_{i=1}^{d_1} e_i e_i^\top \Bigl\{ \sum_{j=1}^{d_0} k_{ij} x_j \bar{x}_j \Bigr\},
        \sum_{i=1}^{d_1} e_i e_i^\top \Bigl\{ \sum_{j=1}^{d_0} c_{ij} x_j x_j \Bigr\}
    \Bigr)
    = \bigotimes_{i=1}^{d_1}
        \mathcal{CN}\Bigl(
            \sum_{j=1}^{d_0} \theta_{ij} x_j,
            \sum_{j=1}^{d_0} k_{ij} x_j \bar{x}_j,
            \sum_{j=1}^{d_0} c_{ij} x_j x_j
        \Bigr)
    \,. $$

Let's suppose that the complex random vector $\mathop{vec}W$ is proper, i.e. $C = 0$.
Then $y$ is itself proper, has independent components and
$$
    y_i \sim \mathcal{CN}\Bigl(
            \sum_{j=1}^{d_0} \theta_{ij} x_j,
            \sum_{j=1}^{d_0} k_{ij} \lvert x_j \rvert^2, 0
        \Bigr)
    \,. $$
In a form, more aligned with `the reparametrization trick`, the expression for the
output is
$$
    y_i
        = \sum_{j=1}^{d_0} \theta_{ij} x_j
        + \sqrt{\sum_{j=1}^{d_0} k_{ij} \lvert x_j \rvert^2}
            \varepsilon_i
    \,,
    \varepsilon_i \sim \mathcal{CN}\Bigl(
            0, 1, 0
        \Bigr)
    \,. $$

Observe that if $z \sim \mathcal{CN}(\mu, \gamma, 0)$ for $\gamma \in \mathbb{C}$, then
$\gamma = \gamma^H = \bar{\gamma}$, $\gamma\in \mathbb{R}$ and $\gamma \geq 0$:
$$
    \begin{pmatrix}\Re z \\ \Im z\end{pmatrix}
    = \begin{pmatrix}\Re \mu \\ \Im \mu \end{pmatrix}
    + \sqrt{\gamma} \varepsilon
    \,,
        \varepsilon \sim \mathcal{N}_{2} \bigl( 0, \tfrac12 I\bigr)
    \,,
$$

The reparametrization $z_\omega = \theta_\omega \varepsilon_\omega$ for
$\varepsilon_\omega \sim \mathcal{CN}(1, \alpha_\omega, 0)$.

$$
    z_\omega
        = \theta_\omega \varepsilon_\omega
    \,,
    \varepsilon_\omega
        \sim \mathcal{CN}(1, \alpha_\omega, 0)
    \Leftrightarrow
    z_\omega
        \sim \mathcal{CN}(\theta_\omega, \alpha_\omega \lvert \theta_\omega\rvert^2, 0)
    \Leftrightarrow
    z_\omega
        = \theta_\omega + \sigma_\omega \varepsilon_\omega
    \,, \sigma_\omega^2
        = \alpha_\omega \lvert \theta_\omega\rvert^2
    \,,
    \varepsilon_\omega
        \sim \mathcal{CN}(0, 1, 0)
    \,. $$

<br>

### Entropy and divergences

The entropy of a generic gaussian random vector $x\sim q(x) = \mathcal{N}_d(x\mid\mu ,\Sigma)$
is
$$
    \mathbb{E}_{x\sim q} \log q(x)
        = - \tfrac{d}2\log 2\pi - \tfrac12 \log\det\Sigma
        - \tfrac12 \mathbb{E}_{x\sim q}\mathop{tr}{\Sigma^{-1} (x-\mu)(x-\mu)^\top}
        = - \tfrac12 \log\det 2\pi e \Sigma
    \,. $$

Therefore the entropy of a gaussian complex random vector $z\sim \mathcal{CN}_d(\theta, K, C)$
is exaclty the entropy of the $2d$ double-real gaussian vector with a special
convariance structure: for $z\sim q(z) = \mathcal{CN}_d(z\mid \theta, K, C)$
$$
    \mathbb{E}_{z\sim q} \log q(z)
        = - \tfrac12 \log\det \pi e \begin{pmatrix}
                 \Re (K + C) & \Im (- K + C) \\
                 \Im (K + C) & \Re (K - C)
            \end{pmatrix}
        = - \tfrac{2d}2 \log\pi e
        - \tfrac12 \log\det \begin{pmatrix}
                 \Re (K + C) & \Im (- K + C) \\
                 \Im (K + C) & \Re (K - C)
            \end{pmatrix}
    \,. $$
If $C=0$, i.e. the complex vector is proper, then
$$
    \det \begin{pmatrix}
         \Re (K + C) & \Im (- K + C) \\
         \Im (K + C) & \Re (K - C)
    \end{pmatrix}
        = \det \begin{pmatrix}
             \Re K & - \Im K \\
             \Im K & \Re K
        \end{pmatrix}
        = \det \hat{K}
        = \det K \overline{\det K}
        = \lvert \det K \rvert^2
        \,, $$
whence the differential entropy becomes
$$
    \mathbb{E}_{z\sim q} \log q(z)
        = - \log (\pi e)^d\lvert \det K\rvert
        = - \log \lvert \det \pi e K\rvert
    \,. $$

<br>

Let $q$ be any distribution on $z$: a product with block terms, or even dirac $\delta_{z_*}$
distributions (for MLE), a mixture or whatever. $q$ may depend on anything, even $\theta$!

Then
$$
    \log p(D; \theta)
        % = \mathbb{E}_{z\sim q} \log p(D; \theta)
        % = \mathbb{E}_{z\sim q} \log \tfrac{p(D, z; \theta)}{p(z\mid D; \theta)}
        = \underbrace{
            \mathbb{E}_{z\sim q} \log \tfrac{p(D, z; \theta)}{q(z)}
        }_{\mathfrak{L}(\theta, q)}
        + \underbrace{
            \mathbb{E}_{z\sim q} \log \tfrac{q(z)}{p(z\mid D; \theta)}
        }_{KL(q \| p(\cdot \mid D; \theta))}
        = \mathbb{E}_{z\sim q} \log p(D\mid z; \theta)
        - \mathbb{E}_{z\sim q} \log \tfrac{q(z)}{p(z)}
        + \mathbb{E}_{z\sim q} \log \tfrac{q(z)}{p(z\mid D; \theta)}
    \,, $$
is constant w.r.t. $q$ at any $\theta$. Since KL-divergence is nonnegative (from
Jensen's inequality), the Evidence Lower Bound $\mathfrak{L}(\theta, q)$ bound the
original log-likelihood $\log p(D; \theta)$ from below.

Let's maximize the ELBO with respec to $q$ and $\theta$. However, since the whole
right hand side is constant with respect to $q$, maximization of $\mathfrak{L}(\theta, q)$
w.r.t. $q \in \mathcal{F}$ (holding $\theta$ fixed) is equivalent to minimizing
$KL(q \| p(\cdot \mid D; \theta))$ w.r.t. $q$. This is the E-step of the EM.

The M-step is maximizing the ELBO over $\theta$ holding $q$ fixed. Note that some of
$q$ may be `offlaid` to the M-step from the E-step!

<br>

Suppose that the prior $p(z)$ and the variational distribution are fully factorized:
$p(z) = \otimes_{\omega} p(z_\omega)$, $q(z) = \otimes_{\omega} q(z_\omega)$. Then
$$
    \mathbb{E}_{z\sim q} \log \tfrac{q(z)}{p(z)}
        = \sum_\omega \mathbb{E}_{z\sim q} \log \tfrac{q(z_\omega)}{p(z_\omega)}
        = \sum_\omega \mathbb{E}_{q(z_\omega)} \log \tfrac{q(z_\omega)}{p(z_\omega)}
    \,. $$
<span style="color:red">**NOTE**</span> we treat $p$ and $q$ as symbols, represending
the density of the argument random variable w.r.t. some carrier measure.

$$
    z_\omega
        = \theta_\omega \varepsilon_\omega
    \,,
    \varepsilon_\omega
        \sim \mathcal{CN}(1, \alpha_\omega, 0)
    \Leftrightarrow
    z_\omega
        \sim \mathcal{CN}(\theta_\omega, \alpha_\omega \lvert \theta_\omega\rvert^2, 0)
    \,. $$

Suppose $q(z) = \mathcal{CN}(\theta, \alpha \lvert\theta\rvert^2, 0)$ and
$p(z) \propto \lvert z\rvert^{-\beta}$. Each term in the sum
$$
\begin{align}
    KL(q\|p)
        &= \mathbb{E}_{q(z)} \log \tfrac{q(z)}{p(z)}
        = \mathbb{E}_{q(z)} \log q(z) - \mathbb{E}_{q(z)} \log p(z)
        \\
        &= - \log \bigl\lvert \pi e \alpha \lvert\theta\rvert^2 \bigr\rvert
        + \beta \mathbb{E}_{q(z)} \log \lvert z\rvert
        + C
        \\
        &= - \log \pi e
        - \log \alpha \lvert\theta\rvert^2
        + \beta \mathbb{E}_{\varepsilon \sim \mathcal{CN}(1, \alpha, 0)}
            \log \lvert \theta \rvert \lvert \varepsilon\rvert
        + C
        \\
        &= - \log \pi e - \log \alpha
        + \tfrac{\beta - 2}2 \log \lvert \theta \rvert^2
        + \beta \mathbb{E}_{\varepsilon \sim \mathcal{CN}(1, \alpha, 0)}
            \log \lvert \varepsilon\rvert
        + C
        \\
        &= - \log \pi e - \log \alpha
        + \tfrac{\beta - 2}2 \log \lvert \theta \rvert^2
        + \tfrac\beta2 \mathbb{E}_{z \sim \mathcal{CN}(0, \alpha, 0)}
            \log \bigl\lvert z + 1 \bigr\rvert^2
        + C
        \\
        &= - \log \pi e - \log \alpha
        + \tfrac{\beta - 2}2 \log \lvert \theta \rvert^2
        + \tfrac\beta2 \mathbb{E}_{\varepsilon \sim \mathcal{N}_2\bigl(0, \tfrac\alpha2 I\bigr)}
            \log \bigl((\varepsilon_1 + 1)^2 + \varepsilon_2^2\bigr)
        + C
\end{align}
\,. $$

For $\beta = 2$ the parameter $\theta$ vanishes from the divergence,
and it becomes just the expectation.

Compare this expression to the real variational dropout: for
$q(z) = \mathcal{N}(\theta, \alpha \theta^2)$ and $p(z) \propto \lvert z\rvert^{-1}$
we have
$$
\begin{align}
    KL(q\|p)
        &= \mathbb{E}_{q(z)} \log \tfrac{q(z)}{p(z)}
        = \mathbb{E}_{q(z)} \log q(z) - \mathbb{E}_{q(z)} \log p(z)
        \\
        &= - \tfrac12 \log 2 \pi e - \tfrac12 \log \alpha \theta^2
        + \mathbb{E}_{\xi \sim \mathcal{N}(0, \alpha)}
            \log \bigl\lvert \theta (\xi + 1) \bigr\rvert
        + C
        \\
        &= - \tfrac12 \log 2 \pi e - \tfrac12 \log \alpha
        + \tfrac12 \mathbb{E}_{\xi \sim \mathcal{N}(0, \alpha)}
            \log (\xi + 1)^2
        + C
\end{align}
\,. $$

<hr>

<br>

# trunk

[Complex-Valued Random Variable](https://www.mins.ee.ethz.ch/teaching/wirelessIT/handouts/miscmath1.pdf)

[Caution: The Complex Normal Distribution!](http://www.ee.imperial.ac.uk/wei.dai/PaperDiscussion/Material2014/ReadingGroup_May.pdf)

A complex Gaussian random vector $z\in \mathbb{C}^d$ is completely determiend by the
distribution of $\begin{pmatrix}\Re z \\ \Im z\end{pmatrix} \in \mathbb{R}^{[2\times d]}$
or equivalently a $\mathbb{C}^{[2\times d]}$ vector:
$$
    \begin{pmatrix}z \\ \bar{z} \end{pmatrix}
        = \underbrace{\begin{pmatrix}I & iI \\ I & -iI \end{pmatrix}}_{\Xi}
        \begin{pmatrix}\Re z \\ \Im z\end{pmatrix}
    \,. $$
For a rv $z$ this vector is called the augmented $z$ vector.

Now the transjugation (Hermitian transpose) of $\Xi$ is 
$$
    \Xi^H
        = \begin{pmatrix}I & I \\ -iI & iI \end{pmatrix}
    \,, $$
and $ \Xi^H \Xi = \Xi \Xi^H = \begin{pmatrix} 2I & 0 \\ 0 & 2I \end{pmatrix}$.
Therefore, $\Xi^{-H} = \tfrac12 \Xi$ and $\Xi^{-1} = \tfrac12 \Xi^H$.
Thus
$$
    \begin{pmatrix}\Re z \\ \Im z\end{pmatrix}
        = \Xi^{-1} \begin{pmatrix}z \\ \bar{z} \end{pmatrix}
        = \tfrac12 \Xi^H \begin{pmatrix}z \\ \bar{z} \end{pmatrix}
    \,. $$

Consider the complex covariance of the augmented $z$:
$$
    \mathbb{E}
        \begin{pmatrix}z \\ \bar{z} \end{pmatrix}
        \begin{pmatrix}z \\ \bar{z} \end{pmatrix}^H
        = \begin{pmatrix}
            \mathbb{E} z \bar{z}^\top & \mathbb{E} z z^\top \\
            \mathbb{E} \bar{z} \bar{z}^\top & \mathbb{E} \bar{z} z^\top
        \end{pmatrix}
        = \begin{pmatrix} K & C \\ \bar{C} & \bar{K} \end{pmatrix}
    \,, $$
since effectively $z^H = (\bar{z})^\top = \overline{\bigl(z^\top\bigr)}$.

Since $$
    \bigl(z z^\top\bigr)^H
        = \overline{\bigl(z z^\top\bigr)}^\top
        = \bigl(\bar{z} \bar{z}^\top\bigr)^\top
        = \bar{z}^{\top\top} \bar{z}^\top
        = \bar{z} \bar{z}^\top
    \,, $$
we have $\bar{C} = C^H$. This implies the following constraints on $K$ (the
covariance) and $C$ (the pseudo-covariance): $K \succeq 0$, $K = K^H$ and
$C=C^\top$.

This complex covariance matrix corresponds to the following real-vector covariance:
$$
    \mathbb{E}
        \begin{pmatrix}\Re z \\ \Im z\end{pmatrix}
        \begin{pmatrix}\Re z \\ \Im z\end{pmatrix}^\top
        = \mathbb{E}
            \begin{pmatrix}\Re z \\ \Im z\end{pmatrix}
            \begin{pmatrix}\Re z \\ \Im z\end{pmatrix}^H
        = \Xi^{-1}
            \begin{pmatrix} K & C \\ \bar{C} & \bar{K} \end{pmatrix}
        \Xi^{-H}
        = \tfrac14 \Xi^H \begin{pmatrix}
            K & C \\ \bar{C} & \bar{K}
        \end{pmatrix} \Xi  % \begin{pmatrix}I & iI \\ I & -iI \end{pmatrix}
%         = \tfrac14 \begin{pmatrix}I & I \\ -iI & iI \end{pmatrix}
%         \begin{pmatrix}
%             K + C & i (K - C) \\ \bar{C} + \bar{K} & i(\bar{C} - \bar{K})
%         \end{pmatrix}
%         = \tfrac14 \begin{pmatrix}
%              K + \bar{K} + C + \bar{C}
%                  & i (K - \bar{K} - (C - \bar{C}))
%                      \\
%              - i (K - \bar{K} + C - \bar{C})
%                  & K + \bar{K} - (C + \bar{C})
%         \end{pmatrix}
        = \tfrac12 \begin{pmatrix}
             \Re (K + C) & - \Im (K - C) \\
             \Im (K + C) & \Re (K - C)
        \end{pmatrix}
        = \tfrac12 \begin{pmatrix}
             \Re (K + C) & \Im (\overline{K - C}) \\
             \Im (K + C) & \Re (\overline{K - C})
        \end{pmatrix}
    \,, $$
since $- \Im z = \Im \bar{z}$ and $\Re z = \Re \bar{z}$.

$$ \Im (K+C)^\top
    = \Im (K^\top+C^\top)
    = \Im (\bar{K} + C)
    = \Im \bar{K} + \Im C
    = -\Im K + \Im C
    = -\Im (K - C)
    \,. $$

$$ \Re (K-C)^\top
    = \Re (K^\top - C^\top)
    = \Re (\bar{K} - C)
    = \Re \bar{K} - \Re C
    = \Re K - \Re C
    = \Re (K - C)
    \,. $$

<br>

<span style="color:red;">**TODO**</span>
Futhermore, the whole matrix must be complex positive semi-definite.
$$
    \begin{pmatrix}a \\ b \end{pmatrix}^\top
        \begin{pmatrix}
             \Re (K + C) & - \Im (K - C) \\
             \Im (K + C) & \Re (K - C)
        \end{pmatrix}
    \begin{pmatrix}a \\ b \end{pmatrix}
        = \tfrac14 \begin{pmatrix}a \\ b \end{pmatrix}^H \Xi^H
            \begin{pmatrix}
                K & C \\ \bar{C} & \bar{K}
            \end{pmatrix}
        \Xi \begin{pmatrix}a \\ b \end{pmatrix}
        = \tfrac14 \begin{pmatrix}a + i b \\ a - i b \end{pmatrix}^H
            \begin{pmatrix}
                K & C \\ \bar{C} & \bar{K}
            \end{pmatrix}
        \begin{pmatrix}a + i b \\ a - i b \end{pmatrix}
        = \tfrac14 \begin{pmatrix}z \\ \bar{z} \end{pmatrix}^H
            \begin{pmatrix}
                K & C \\ \bar{C} & \bar{K}
            \end{pmatrix}
        \begin{pmatrix}z \\ \bar{z} \end{pmatrix}
%         = \tfrac14 \begin{pmatrix}z \\ \bar{z} \end{pmatrix}^H
%             \begin{pmatrix}
%                 K z + C \bar{z} \\ \bar{C} z + \bar{K} \bar{z}
%             \end{pmatrix}
%         = \tfrac14 \bigl(
%                 z^H K z + z^H C \bar{z} + \bar{z}^H \bar{C} z + \bar{z}^H \bar{K} \bar{z}
%         \bigr)
        = \tfrac14 \bigl(
                z^H K z + z^H C \bar{z} + \overline{z^H C \bar{z}} + \overline{z^H K z}
        \bigr)
        = \tfrac12 \Re (z^H K z) + \tfrac12 \Re (z^H C \bar{z})
    \,. $$

<br>

Let $A$ and $B$ be conforming $\mathbb{C}$-valued matrices. Let
$$
    \hat{\cdot}
    \colon \mathbb{C}^{n \times m} \to \mathbb{R}^{2n \times 2m}
    \colon A \mapsto \begin{pmatrix}
            \Re A & - \Im A \\
            \Im A & \Re A \\
        \end{pmatrix}
    \,. $$
$$
\begin{align}
    AB = \Re A \Re B - \Im A \Im B + i (\Re A \Im B + \Im A \Re B)
    &\Leftrightarrow
        \begin{pmatrix}
            \Re A & - \Im A \\
            \Im A & \Re A \\
        \end{pmatrix}
        \begin{pmatrix}
            \Re B & - \Im B \\
            \Im B & \Re B \\
        \end{pmatrix}
        = \begin{pmatrix}
            \Re A \Re B - \Im A \Im B & - (\Re A \Im B + \Im A \Re B) \\
            \Im A \Re B + \Re A \Im B & \Re A \Re B - \Im A \Im B\\
        \end{pmatrix}
        \,, \\
    \widehat{AB} &= \hat{A} \hat{B}
        \,, \\
    \widehat{A^H} &= \hat{A}^\top
        \,, \\
    I_{2n}
        = \widehat{I}
        = \widehat{A A^{-1}}
        = \hat{A} \widehat{A^{-1}}
        &\Rightarrow
        \widehat{A^{-1}} = (\hat{A})^{-1}
        \,, \\
    \det \hat{A}
        = \det \begin{pmatrix}
            I & i I \\
            0 & I \\
        \end{pmatrix}
        \begin{pmatrix}
            \Re A & - \Im A \\
            \Im A & \Re A \\
        \end{pmatrix}
        \begin{pmatrix}
            I & - i I \\
            0 & I \\
        \end{pmatrix}
%         = \det \begin{pmatrix}
%             \Re A + i \Im A & - \Im A + i \Re A \\
%             \Im A & \Re A \\
%         \end{pmatrix}
%         \begin{pmatrix}
%             I & - i I \\
%             0 & I \\
%         \end{pmatrix}
        &= \det \begin{pmatrix}
            A & i A \\
            \Im A & \Re A \\
        \end{pmatrix}
        \begin{pmatrix}
            I & - i I \\
            0 & I \\
        \end{pmatrix}
        = \det \begin{pmatrix}
            A & 0 \\
            \Im A & \bar{A} \\
        \end{pmatrix}
        = \det A \det \bar{A}
        = \det A \overline{\det A}
        \,.
\end{align}
$$

On vectors the hat-operator works by treating vectors as one-column matrices.
$$
    \widehat{u^H v}
        = \widehat{u^H} \hat{v}
        = \hat{u}^\top \hat{v}
        = \begin{pmatrix}
            \Re u^\top & \Im u^\top \\
            - \Im u^\top & \Re u^\top
        \end{pmatrix} \begin{pmatrix}
            \Re v & - \Im v \\
            \Im v & \Re v
        \end{pmatrix}
        = \begin{pmatrix}
            \Re u^\top \Re v + \Im u^\top \Im v
                & - (\Re u^\top \Im v - \Im u^\top \Re v) \\
            \Re u^\top \Im v - \Im u^\top \Re v
                & \Im u^\top \Im v + \Re u^\top \Re v
        \end{pmatrix}
    \,. $$

If $Q\in \mathbb{C}^{n\times n}$ is postive semidefinite, then $z^H Q z \geq 0$ for all $z\in \mathbb{C}^n$.
Then
$$
    0 \leq z^H Q z
        = \Re z^H Q z
        \equiv \begin{pmatrix}
            \Re z^H Q z & 0 \\
            0 & \Re z^H Q z
        \end{pmatrix}
        = \widehat{z^H Q z}
        = \widehat{z^H} \widehat{Q z}
        = \hat{z}^\top \hat{Q} \hat{z}
    \,. $$

<hr>

$$
    \mathbb{E}_{\xi \sim \mathcal{N}(0, \alpha)}
        \log (\xi + 1)^2
            = \log \alpha
            + \mathbb{E}_{\xi \sim \mathcal{N}(0, 1)}
                \log (\xi + \tfrac1{\sqrt\alpha})^2
            = \log \alpha + g\bigl(\tfrac1{\sqrt\alpha}\bigr)
    \,. $$

$$
    g(\mu)
        = \mathbb{E}_{\xi \sim \mathcal{N}(0, 1)} \log (\xi + \mu)^2
\,. $$

$$\begin{align}
    \int_{-\infty}^\infty
        \tfrac1{\sqrt{2\pi}} e^{-\tfrac12x^2}
        \log \lvert x + \mu\rvert dx
        &= \int_{-\infty}^{-\mu}
            \tfrac1{\sqrt{2\pi}} e^{-\tfrac12x^2}
            \log (- x - \mu) dx
        + \int_{-\mu}^\infty
            \tfrac1{\sqrt{2\pi}} e^{-\tfrac12x^2}
            \log (x + \mu) dx
        \\
        &= \int_\mu^\infty
            \tfrac1{\sqrt{2\pi}} e^{-\tfrac12x^2}
            \log (x - \mu) dx
        + \int_{-\mu}^\infty
            \tfrac1{\sqrt{2\pi}} e^{-\tfrac12x^2}
            \log (x + \mu) dx
\end{align}$$

Supppose $x \sim \chi^2_k$, the cumulant function of $y = \log x$ is
$$
    K(t)
        = \log \mathbb{E} e^{ty}
        = \log \mathbb{E} x^t
    \,, $$
which is known to be
$$
    K(t)
        = t \log2 + \log\Gamma\bigl(\tfrac{k}2 + t\bigr) - \log \Gamma\bigl(\tfrac{k}2\bigr)
    \,. $$
Now
$$
    \mathbb{E} y^k
        != \tfrac{d^k}{dt^k} K(t) \big\vert_{t=0}
    \,. $$

<br>